# Thinkful Challenge: Data Cleaning & Validation

#### In this challenge, I will use this dataset of article open-access prices paid by the WELLCOME Trust between 2012 and 2013. To complete this challenge, I will determine the five most common journals and the total articles for each. I will also calculate the mean, median, and standard deviation of the open-access cost for each journal. The data will have to be cleaned thoroughly in order to extract accurate estimates, and I will describe the steps I have taken below.


First we'll import all of the packages we need. Then we can take a look at what we're working with by reading in the CSV and printing out a few lines.

In [ ]:
import pandas as pd
import numpy as np
import statistics as stats

import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


In [2]:
main_df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding="latin-1")         # Read in the CSV file

main_df.columns = ['PMID', 'Publisher', 'Journal', 'Article', "Cost"]                      # Rename some columns for ease

main_df.head(10)                                                                           # Display the first 10 rows

,PMID,Publisher,Journal,Article,Cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,£2367.95
7,23057412 PMC3495574,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,£649.33
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,£1294.78


We are aiming to use this data set to find statistics related to the journals, so if the journal title is not included at all, the other pieces of data are not useful for our specific research. Let's drop anything matching that criteria.

In [3]:
main_df = main_df.dropna(subset=['Journal'])


Now we'll define a function that will get rid of all the pesky empty space that could be lurking inside the many strings of our data.

In [4]:
def trim_all_columns(data_frame_in):
    
    # Trim whitespace from ends of each value across all series in dataframe, given that it's a string
    trim_strings = lambda x: x.strip() if type(x) is str else x
    return data_frame_in.applymap(trim_strings)


main_df = trim_all_columns(main_df)

I also noticed there are some cost values that don't make sense, such as £999999.00 or a number with an American dollar sign. The documentation says all these costs were already converted to £ sterling so we will assume we can simply drop all signs and any amounts that are clearly inaccurate

In [5]:
# So let's take out all the currency symbols
main_df['Cost'] = main_df['Cost'].map(lambda x: x.strip('£'))

main_df['Cost'] = main_df['Cost'].map(lambda y: y.strip('$'))


Now we can easily get rid of the costs that are equal to '999999.00', of which there are multiple. We will replace them with the mean value of the data set as it's likely they weren't meant to be = 0, but we also don't want to get rid of them altogether.

In [6]:
# So first, convert the cost column into numbers, and sort them in descending order
main_df['Cost'] = pd.to_numeric(main_df.Cost)
main_df = main_df.sort_values(by=['Cost'], ascending=False)

# Now calculate the mean of the values that are not 999999 (there are 47 of these cases)
cost_avg = round(main_df['Cost'][47:].mean(), 2)

# Then replace all of the incorrect 999999 costs with the mean
main_df['Cost'].replace(999999.0, cost_avg, inplace=True)



Now that we've cleaned up the cost column, it's time to clean up the other column that we'll be using to calculate the specific statistics that are being asked for. Thus, we will begin cleaning the "Journal Title" column.

In [7]:
# Start off by replacing characters we don't necessarily need, such as colons, periods, parenthesis, dashes, etc
main_df['Journal'] = main_df['Journal'].str.replace(':', '')

main_df['Journal'] = main_df['Journal'].str.replace(' - ', ' ')
main_df['Journal'] = main_df['Journal'].str.replace('--', '-')
main_df['Journal'] = main_df['Journal'].str.replace('-', ' ')

main_df['Journal'] = main_df['Journal'].str.replace("'", "")

main_df['Journal'] = main_df['Journal'].str.replace('.', '')

main_df['Journal'] = main_df['Journal'].str.replace('(', '')
main_df['Journal'] = main_df['Journal'].str.replace(')', '')

main_df['Journal'] = main_df['Journal'].str.replace(',', '')

main_df['Journal'] = main_df['Journal'].str.replace('   ', ' ')
main_df['Journal'] = main_df['Journal'].str.replace('  ', ' ')

# I'm also going to replace ampersands (&) with the actual word "and"
main_df['Journal'] = main_df['Journal'].str.replace('&', 'and')

# Then I'll sort everything by the Journal name to help me look through it
main_df = main_df.sort_values(by=['Journal'], ascending=True)

Following this we will capitalize everything to make it easier to clean the data. This way if two things are spelled exactly the same but have different cases they will now be equivalent.

In [8]:
main_df['Journal'] = main_df['Journal'].str.upper() 


Now I'm going to use the fuzzywuzzy package to standardize the names of journals via more replacement. I do this by making a list of unique values and iterating through them to fix the dataframe. This way, by the end of the loop it should have packaged each unique journal into one name.

In [9]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match


all_strings = main_df['Journal'].unique()

for each_name in all_strings:
    replace_matches_in_column(df=main_df, column='Journal', string_to_match=each_name)


Now we are ready create our descriptive statistics. First we'll drop the columns that we won't need for right now.

In [10]:
main_df = main_df.drop(columns="PMID")
main_df = main_df.drop(columns="Publisher")
main_df = main_df.drop(columns="Article")


Following this, we can show the number of times each journal comes up and display this number. We will show the top 5 journals for now.

In [11]:
# Count it
count_df = main_df['Journal'].value_counts()

# Put it back into a dataframe
count_df = count_df.to_frame()

# Change the column name for ease of reading
count_df.columns = ['Journal Frequency']

# Show it
count_df.head(5)



,Journal Frequency
PLOS ONE,191
JOURNAL BIOLOGICAL CHEMISTRY,61
NEUROLMAGE,31
NUCLEIC ACID RESEARCH,29
PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES PNAS,25


After all that, we can now show the other summary statistics we listed above. First I will calculate the sum of all costs attributed to each journal and display the top 10.

In [12]:
# Group our data by the journal name and sum their total costs
sum_df = main_df.groupby('Journal')['Cost'].sum()

# Change this output back into a dataframe so that we can show it easily and print out the highest total costs
sum_df = sum_df.to_frame()

sum_df = sum_df.sort_values(by=['Cost'], ascending=False)

sum_df.head(10)

,Cost
Journal,
PLOS ONE,377670.48
MOVEMENT DISORDERS,227651.82
JOURNAL BIOLOGICAL CHEMISTRY,86485.16
NEUROLMAGE,68577.62
NATURE COMMUNICATIONS,58424.06
DEVELOPMENTAL CELL,43137.90
PLOS GENETICS,40167.76
PLOS NEGECTED TROPICAL DISEASES,40041.25
HUMAN MOLECULAR GENETICS,38748.41


Finally, we can calculate an accurate mean, median, and standard deviation for each journal.

In [13]:
full_df_mean = round(main_df['Cost'].mean(), 2)
print("The mean of all costs is: " + str(full_df_mean))

full_df_med = round(main_df['Cost'].median(), 2)
print("The median of all costs is: " + str(full_df_med))

full_df_sdv = round(main_df['Cost'].std(), 2)
print("The standard deviation of all costs is: " + str(full_df_sdv))

The mean of all costs is: 2009.66
The median of all costs is: 1883.86
The standard deviation of all costs is: 6029.15
